## Preliminary:  Importing libraries and defining known values
I've started by importing the relevant libraries.

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

Further, i've initialized all the known entities. These are mentioned below:

1- URL: 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

2- table_attribs: The attributes or column names for the dataframe stored as a list. Since the data available in the website is in USD Millions, the   a ttributes should initially be 'Country' and 'GDP_USD_millions'. This will be modified in the transform function later.

3- db_name: As mentioned in the Project scenario, 'World_Economies.db'

4- table_name: As mentioned in the Project scenario, 'Countries_by_GDP'

5- csv_path: As mentioned in the Project scenario, 'Countries_by_GDP.csv'

In [5]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

## Task 1: Extracting information
Notes : 

Inspecting the URL and note the position of the table i've found that even the images with captions in them are stored in tabular format. Hence, in the given webpage, our table is at the third position, or index 2. Among this, we require the entries under 'Country/Territory' and 'IMF -> Estimate'.

Note that there are a few entries in which the IMF estimate is shown to be '—'. Also, there is an entry at the top named 'World', which we do not require. Segregate this entry from the others because this entry does not have a hyperlink and all others in the table do. So i've taken advantage of that and accessed only the rows for which the entry under 'Country/Terriroty' has a hyperlink associated with it.

Note that '—' is a special character and not a general hyphen, '-'. 

In [24]:
def extract(url, table_attribs):
    
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    
    # Extracting the html content of web page as text
    page = requests.get(url).text

    # Parse the text into an HTML object.
    data = BeautifulSoup(page,'html.parser')

    # Create an empty pandas DataFrame named df with columns as the table_attribs
    df = pd.DataFrame(columns=table_attribs)

    # Extract all 'tbody' attributes of the HTML object : <tbody> contains the data of the table body
    tables = data.find_all('tbody')

    # Extract all the rows of the index 2 table using the 'tr' attribute : <tr> tag contains the data within the row
    rows = tables[2].find_all('tr')

    
    for row in rows:
        # Check the contents of each row, having attribute ‘td’ : <td> tag contains the data within the each cell of the row
        col = row.find_all('td')

        # The row should not be empty.
        if len(col)!=0:

            # The first column should contain a hyperlink & The third column should not be '—'.
            if col[0].find('a') is not None and '—' not in col[2]:

                # Extract the IMF estimation of GDP
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                
                # Store all entries matching the conditions in step 5 to a dictionary with keys the same as entries of table_attribs. 
                df1 = pd.DataFrame(data_dict, index=[0])

                # Append all these dictionaries one by one to the dataframe.
                df = pd.concat([df,df1], ignore_index=True)

    return df

inspecting the result

In [31]:
df = extract(url, table_attribs)
df

,Country,GDP_USD_millions
0,United States,"26,854,599"
1,China,"19,373,586"
2,Japan,"4,409,738"
3,Germany,"4,308,854"
4,India,"3,736,882"
...,...,...
186,Marshall Islands,291
187,Palau,262
188,Kiribati,248
189,Nauru,151


## Task 2: Transform information

In [26]:
def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''

    # Extracting the dataframe values to a list
    GDP_list = df["GDP_USD_millions"].tolist()

    # Converting the contents of the 'GDP_USD_millions' column of df dataframe from currency format to floating numbers.  
    GDP_list = [float("".join(x.split(','))) for x in GDP_list]

    # Dividing all these values by 1000 and round it to 2 decimal places.
    GDP_list = [np.round(x/1000,2) for x in GDP_list]

    # Applying the changes on the dataframe
    df["GDP_USD_millions"] = GDP_list

    # Modifying the name of the column from 'GDP_USD_millions' to 'GDP_USD_billions'.
    df=df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})


    return df

Inscpecting the result

In [32]:
df = transform(df)
df

,Country,GDP_USD_billions
0,United States,26854.60
1,China,19373.59
2,Japan,4409.74
3,Germany,4308.85
4,India,3736.88
...,...,...
186,Marshall Islands,0.29
187,Palau,0.26
188,Kiribati,0.25
189,Nauru,0.15


## Task 3: Loading information

In [29]:
def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''

    df.to_csv(csv_path)


In [34]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''

    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

## Task 4: Querying the database table

In [37]:
def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
  
    print(query_statement)

    # Running the query statement on the table and retrieving the output as a filtered dataframe.
    query_output = pd.read_sql(query_statement, sql_connection)

    # Printing the dataframe
    print(query_output)



## Task 5: Logging progress

In [39]:
def log_progress(message):
# Here, message text is passed to the function as an argument. Each entry must be in a separate line.
    
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''

    ''' Here, you define the required entities and call the relevant 
        functions in the correct order to complete the project. Note that this
        portion is not inside any function.'''

    # Year-Monthname-Day-Hour-Minute-Second 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 

    # get current timestamp 
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 

    # Creating and writing in the logs file
    with open("./etl_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

## Function calls

In [40]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
